In [1]:
# imports 
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import re
import json
import csv
from urllib.parse import quote
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
def get_movie_url(year):
    url = ('https://www.imdb.com/search/title/?title_type=feature&boxoffice_gross_us=1,&release_date='+str(year)+'-01-01,'+str(year)+'-12-31&countries=us&view=simple&count=250')
    url2 = ('https://www.imdb.com/search/title/?title_type=feature&boxoffice_gross_us=1,&&release_date='+str(year)+'-01-01,'+str(year)+'-12-31&countries=us&view=simple&count=250&start=251')
    headers = {'Accept-Language': 'en-US'}
    movies_html = requests.get(url.format(), headers=headers).content
    movies2_html = requests.get(url2.format(), headers=headers).content
    soup = BeautifulSoup(movies_html, 'html.parser')
    soup2 = BeautifulSoup(movies2_html, 'html.parser')
    soup_main = soup.find("div", {"id": "main"})
    soup_main2 = soup2.find("div", {"id": "main"})
    movies = soup_main.findAll('a', {'href': re.compile('/title/tt')})
    movies2 = soup_main2.findAll('a', {'href': re.compile('/title/tt')})
    # create an array to house all **UNIQUE** ids for each movie
    ids_for_movies_in_year = []
    for movie in movies:
        # Getting the movie attributes, splitting them on '/'
        title_id = movie.attrs['href'].split('/')[2]
        # Here is where we are saying "does this ID already exist in our array? No? OK add it!"
        if title_id not in ids_for_movies_in_year: 
            ids_for_movies_in_year.append(title_id)
            # Testing
            # print(movie.attrs['href'].split('/')[2])
    for movie in movies2:
        # Getting the movie attributes, splitting them on '/'
        title_id = movie.attrs['href'].split('/')[2]
        # Here is where we are saying "does this ID already exist in our array? No? OK add it!"
        if title_id not in ids_for_movies_in_year: 
            ids_for_movies_in_year.append(title_id)
            # Testing
            # print(movie.attrs['href'].split('/')[2])
    return(ids_for_movies_in_year)

all_the_ids = []
for year in range(1970, 2020):
    all_the_ids.append((get_movie_url(year)))

all_the_ids_df = pd.DataFrame(all_the_ids)
all_the_ids_df.to_csv(r'imdb_ids.csv')

all_the_2018_ids_df = pd.DataFrame(all_the_ids[48])
all_the_2018_ids_df.to_csv(r'imdb_ids_2018.csv')

In [3]:
def write_csv(data):
    df = pd.DataFrame(data)
    df.to_csv('2018_movies.csv', index=False)

headers = {'Accept-Language': 'en-US'}
payload = "{}"

api_key = open("tmdb_api_key.txt")
api_key = api_key.read()

all_movie_data = []
with open('imdb_ids_2018.csv', encoding='utf-8') as csvfile:
    movies = csv.reader(csvfile)
    for movie in movies:
        try:
            url = "https://api.themoviedb.org/3/movie/"
            thing_looking_for = movie[1]
            my_api_key = "?api_key=" + api_key
            full_url = url + thing_looking_for + my_api_key
            res = requests.get(full_url, payload, headers=headers)
            data = res.content.decode('UTF-8')
            jdata = json.loads(data)
            try:
                title = jdata['title']
                budget = jdata['budget']
                genres = jdata['genres']
                production_companies = jdata['production_companies']
                release_date = jdata['release_date']
                revenue = jdata['revenue']
                profit = revenue - budget
                popularity = jdata['popularity']
                vote_average = jdata['vote_average']
                vote_count = jdata['vote_count']
            except KeyError:
                title = 'NA'
                budget = 'NA'
                genres = 'NA'
                production_companies = 'NA'
                release_date = 'NA'
                revenue = 'NA'
                profit = 'NA'
                popularity = 'NA'
                vote_average = 'NA'
                vote_count = 'NA'

            movie_data = {
                'release_date': release_date,
                'title': title,
                'budget': budget,
                'genres': genres,
                'production_companies': production_companies,
                'revenue': revenue,
                'profit': profit,
                'popularity': popularity,
                'vote_average': vote_average,
                'vote_count': vote_count
            }
            all_movie_data.append(movie_data)
        except UnicodeDecodeError:
            director_data = {}
all_movie_data_df = pd.DataFrame(all_movie_data)
print(all_movie_data_df)
write_csv(all_movie_data)

    release_date title budget genres production_companies revenue profit  \
0             NA    NA     NA     NA                   NA      NA     NA   
1             NA    NA     NA     NA                   NA      NA     NA   
2             NA    NA     NA     NA                   NA      NA     NA   
3             NA    NA     NA     NA                   NA      NA     NA   
4             NA    NA     NA     NA                   NA      NA     NA   
..           ...   ...    ...    ...                  ...     ...    ...   
309           NA    NA     NA     NA                   NA      NA     NA   
310           NA    NA     NA     NA                   NA      NA     NA   
311           NA    NA     NA     NA                   NA      NA     NA   
312           NA    NA     NA     NA                   NA      NA     NA   
313           NA    NA     NA     NA                   NA      NA     NA   

    popularity vote_average vote_count  
0           NA           NA         NA  
1    

In [4]:
movies = pd.read_csv('2018_movies.csv')

In [5]:
movies

,release_date,title,budget,genres,production_companies,revenue,profit,popularity,vote_average,vote_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
